# Libraries

In [88]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences

import numpy as np
import os

# Preprocessing

In [89]:
# base path
directory_path = '/mnt/d/GitHub/SSLrecognition/train_data/videos'
# current directory
c_dir = os.getcwd()

# all actions
actions = np.array(sorted([folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))])) # sorted to follow folder arrangement

# specific actions
# actions = np.array(['base', 'again', 'how', 'open', 'sorry', 'see'])

In [90]:
# create a dictionary for int representation of actions
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'afternoon': 0,
 'again': 1,
 'base': 2,
 'door': 3,
 'hello': 4,
 'house': 5,
 'how': 6,
 'open': 7,
 'see': 8,
 'sorry': 9,
 'why': 10}

Note that at this point, we will not access the video folder, only the numpy folder.

In [91]:
sequences, labels = [], []  # sequence -> video, labels -> action
for action in actions:
    no_actions = len(os.listdir(os.path.join(c_dir, 'labels', action)))
    print('Opening path:', os.path.join(c_dir, 'labels', action))
    print(f'Number of instances: {no_actions}')
    for num in range(1, no_actions + 1):
        window = []         # window -> single frame
        file = str(action) + "_" + str(num)
        no_frames_per_action = len(os.listdir(os.path.join(c_dir, 'labels', action, file)))
        print(f'Number of frames in {file}: {no_frames_per_action}')
        for frame_num in range(1, no_frames_per_action + 1):
            res = np.load(os.path.join(c_dir, 'labels', action, file,  "{}.npy".format(frame_num)))     # res -> coordinate key points
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
    print('-'*75)

Opening path: /mnt/d/GitHub/SSLrecognition/train_data/labels/afternoon
Number of instances: 40
Number of frames in afternoon_1: 31
Number of frames in afternoon_2: 30
Number of frames in afternoon_3: 30
Number of frames in afternoon_4: 30
Number of frames in afternoon_5: 31
Number of frames in afternoon_6: 31
Number of frames in afternoon_7: 31
Number of frames in afternoon_8: 30
Number of frames in afternoon_9: 31
Number of frames in afternoon_10: 31
Number of frames in afternoon_11: 31
Number of frames in afternoon_12: 31
Number of frames in afternoon_13: 31
Number of frames in afternoon_14: 31
Number of frames in afternoon_15: 31
Number of frames in afternoon_16: 31
Number of frames in afternoon_17: 31
Number of frames in afternoon_18: 31
Number of frames in afternoon_19: 31
Number of frames in afternoon_20: 31
Number of frames in afternoon_21: 31
Number of frames in afternoon_22: 31
Number of frames in afternoon_23: 31
Number of frames in afternoon_24: 31
Number of frames in aftern

In [428]:
# due to difference in number of frames, pad x and y
x = np.array(pad_sequences(sequences, dtype = 'float', padding = 'post', value = 0))
y = pad_sequences(to_categorical(labels).astype(int), dtype = 'int', padding = 'post', value = -1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, stratify = y)

In [429]:
x_train.shape

(216, 117, 225)

# Models

## LSTM

In [645]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard, TerminateOnNaN, EarlyStopping

from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [646]:
# for logging of data with TensorBoard
# log_dir = os.path.join(c_dir, 'Logs')
# tb_callback = TensorBoard(log_dir = log_dir)

# to end training when failure happens ie. loss == nan
term = TerminateOnNaN()

# to stop training early if there is no change in loss
early = EarlyStopping(monitor = 'loss', patience = 5)

In [647]:
def choose_lstm(n):
    if n == 1:
        model = Sequential()
        model.add(LSTM(64, return_sequences = True, input_shape = (117, 225)))
        model.add(LSTM(64, activation = 'sigmoid'))
        model.add(Dropout(0.1))

        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model

    elif n == 2:
        model = Sequential()
        model.add(LSTM(64, return_sequences = True, activation = "relu", input_shape = (117, 225)))
        model.add(LSTM(128, return_sequences = True, activation = "relu"))
        model.add(LSTM(64, return_sequences = False, activation = "relu"))
        model.add(Dense(64, activation = "relu"))
        model.add(Dense(32, activation = "relu"))
        model.add(Dense(8, activation = "relu"))
        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model

    elif n == 3:
        model = Sequential()
        model.add(LSTM(128, return_sequences = True, activation = "sigmoid", input_shape = (117, 225)))
        model.add(Dropout(0.1))
        model.add(LSTM(64, return_sequences = False, activation = "sigmoid"))
        model.add(Dropout(0.1))
        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model

In [648]:
model = choose_lstm(1)
opt = keras.optimizers.Adam(learning_rate = 0.01)
model.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ['categorical_accuracy'])
model.summary()

Model: "sequential_49"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_71 (LSTM)              (None, 117, 64)           74240     
                                                                 
 lstm_72 (LSTM)              (None, 64)                33024     
                                                                 
 dropout_29 (Dropout)        (None, 64)                0         
                                                                 
 dense_59 (Dense)            (None, 11)                715       
                                                                 
Total params: 107979 (421.79 KB)
Trainable params: 107979 (421.79 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [649]:
model.fit(x_train, y_train, epochs = 2000, validation_split = 0.1, callbacks = [term, early])

Epoch 1/2000
7/7 [==============================] - 4s 267ms/step - loss: 2.3558 - categorical_accuracy: 0.1237 - val_loss: 2.1412 - val_categorical_accuracy: 0.1818
Epoch 2/2000
7/7 [==============================] - 1s 197ms/step - loss: 2.2540 - categorical_accuracy: 0.1495 - val_loss: 2.1755 - val_categorical_accuracy: 0.3182
Epoch 3/2000
7/7 [==============================] - 1s 187ms/step - loss: 2.1936 - categorical_accuracy: 0.1753 - val_loss: 2.1849 - val_categorical_accuracy: 0.1364
Epoch 4/2000
7/7 [==============================] - 1s 195ms/step - loss: 2.1531 - categorical_accuracy: 0.1495 - val_loss: 2.1157 - val_categorical_accuracy: 0.3182
Epoch 5/2000
7/7 [==============================] - 1s 200ms/step - loss: 2.0698 - categorical_accuracy: 0.1495 - val_loss: 2.1656 - val_categorical_accuracy: 0.1818
Epoch 6/2000
7/7 [==============================] - 1s 191ms/step - loss: 2.0278 - categorical_accuracy: 0.1701 - val_loss: 2.2306 - val_categorical_accuracy: 0.1364
Epoc

In [650]:
# take model predictions
res = model.predict(x_test)

1/1 [==============================] - 0s 304ms/step


In [651]:
y_true = np.argmax(y_test, axis = 1).tolist()
y_pred = np.argmax(res, axis = 1).tolist()

In [652]:
print(f'Accuracy: {round(accuracy_score(y_true, y_pred)*100, 1)}%')

Accuracy: 25.0%


In [653]:
# import os
# import shutil

# folder_path = './averaged_np_labels/'
# parent_files = os.listdir(os.path.join(folder_path))
# write_path = './labels/'

# for parent_file in parent_files:
#     parent_path = os.path.join(folder_path, parent_file)
#     export_path = os.path.join(write_path, parent_file)
#     os.mkdir(export_path)
#     print(f'Created new directory: {export_path}')
#     for i in range(1, len(os.listdir(os.path.join(folder_path, parent_file)))+1):
#         new_subfolder = f'{parent_file}_{i}'
#         os.mkdir(os.path.join(export_path, new_subfolder))
#         print(f'Created new subdirectory: {new_subfolder}')

#         source = os.path.join(parent_path, f'video{i}')
#         destination = os.path.join(os.path.join(export_path, new_subfolder))

#         sourcefolder = os.listdir(os.path.join(parent_path, f'video{i}'))
#         for file in sourcefolder:
#             file_to_copy = os.path.join(os.path.join(parent_path, f'video{i}'), file)
#             shutil.copy(file_to_copy, destination)